In [1]:
import scipy as sp
import numpy as np
import keras
import os
import csv
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

Using TensorFlow backend.


In [2]:
def preprocess(X):
    # X_centered = X = np.mean(X, axis=1)
    X_normalized = np.divide(X, 255)
    return X_normalized


def load_data(path, expect_labels=True):
    assert path.endswith('.csv')
    # If a previous call to this method has already converted
    # the data to numpy format, load the numpy directly
    X_path = path[:-4] + '.X.npy'
    Y_path = path[:-4] + '.Y.npy'
    if os.path.exists(X_path):
        X = np.load(X_path)
        if expect_labels:
            y = np.load(Y_path)
        else:
            y = None
        return preprocess(X), y

    # Convert the .csv file to numpy
    csv_file = open(path, 'r')

    reader = csv.reader(csv_file)

    # Discard header
    row = next(reader)

    y_list = []
    X_list = []

    for row in reader:
        if expect_labels:
            y_str, X_row_str = (row[0], row[1])
            y = int(y_str)
            assert 0 <= y <= 6
            y_list.append(np.eye(7)[y])
        else:
            X_row_str = row[1]
        X_row_strs = X_row_str.split(' ')
        X_row = [float(x) for x in X_row_strs]
        X_list.append(X_row)

    X = np.asarray(X_list).astype('float32')
    if expect_labels:
        y = np.asarray(y_list)
    else:
        y = None

In [3]:
model = Sequential()
model.add(Conv2D(32, (5, 5), input_shape=(48, 48, 1)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (5, 5)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (5, 5)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# flatter for fully-connected layer
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(7))
model.add(Activation('softmax'))

In [4]:
model.compile(loss='binary_crossentropy', optimizer='adam',
              metrics=['accuracy'])

In [5]:
emotion_dict = { 0: 'Angry', 1: 'Disgust', 2: 'Fear', 3: 'Happy', 
                4: 'Sad', 5: 'Surprise', 6: 'Neutral'}

X_all, Y_all = load_data('./data/fer2013/fer2013.csv')
assert len(X_all) == len(Y_all)
assert X_all is not None and Y_all is not None

X_train, X_test, y_train, y_test = train_test_split(X_all, Y_all,
                                                    test_size=0.33,
                                                    random_state=42)

In [6]:
alpha = 0.0001
epochs = 10
batch_size = 128

In [8]:
train_generator = train_datagen.flow_from_directory(
        'data/train',  # this is the target directory
        target_size=(150, 150),  # all images will be resized to 150x150
        batch_size=batch_size,
        class_mode='binary')  # since we use binary_crossentropy loss, we need binary labels
model.fit(x_train, y_train)

NameError: name 'train_datagen' is not defined

In [ ]:
model.save_weights('first_try.h5')